# Reddit: <a href="http://pushshift.io/">PushShift.io</a>

- <a href="https://www.reddit.com/r/inthenews">/r/inthenews</a>
- <a href="https://www.reddit.com/r/news">/r/news</a>
- <a href="https://www.reddit.com/r/savedyouaclick">/r/savedyouaclick</a>

In [1]:
import json
import numpy as np
import pandas as pd
import time

from datetime import datetime
from pandas.io.json import json_normalize
from pprint import pprint
from sys import maxsize
from urllib import request

### Function to get data from PushShift

In [2]:
def get_submissions(subreddit, ts_start, ts_end):
    dfs = dict()  # {cycle: pd.DataFrame}

    cycle = 1  # cycle index
    time_total = 0  # total time
    ts_current = ts_end  # current timestamp = last interval timestamp

    while ts_current > ts_start:
        time_start = time.time()  # start timestamp of the cycle

        print(f'Cycle {cycle:>4d}: {timestamp_to_utc(ts_current)} ...', end=' ')

        try:
            response = request.urlopen(f'https://api.pushshift.io/reddit/submission/search/'\
                                       f'?subreddit={subreddit}&before={ts_current}&size={maxsize}')
            data = response.read()
            data = json.loads(data)
            # data = get_submissions('news', before=ts_current)
        except:
            print('Error encountered!')
            time_end = time.time()  # end timestamp of the cycle
            time_total += int(time_end - time_start)
            continue

        dfs[cycle] = json_normalize(data['data'])
        if len(dfs[cycle]) <= 0:
            print('No items found!')
            time_end = time.time()  # end timestamp of the cycle
            time_total += int(time_end - time_start)
            break
        ts_current = dfs[cycle]['created_utc'].iloc[-1]

        time_end = time.time()  # end timestamp of the cycle
        print(f'{len(dfs[cycle]):>4d} items done in {int(time_end - time_start):>4d} seconds!')

        cycle += 1
        time_total += int(time_end - time_start)

    print(f'\nThe whole process took {time_total} seconds!')
#     print('Last timestamp: {}\n\n'.format(datetime.fromtimestamp(ts_current) \
#                                                   .strftime('%Y-%m-%d %H:%M:%S')))

    return dfs

In [3]:
# %load '../snippets/timestamp_to_utc.py'
def timestamp_to_utc(timestamp):
    return datetime.fromtimestamp(timestamp) \
                   .strftime('%Y-%m-%d %H:%M:%S')

## Set output folder

In [4]:
data_out = '../data/00_raw/pushshift'

## Get submissions

### /r/inthenews, /r/savedyouaclick

#### [01.01.2018 - 08.08.2018]

In [7]:
for year in [2018]:
    for subreddit in ['inthenews', 'savedyouaclick']:  # news
        # timestamp of the start of the interval
        ts_start = datetime.strptime(f'01.01.{year} 00:00:00,00',
                                     '%d.%m.%Y %H:%M:%S,%f')
        ts_start = int(ts_start.timestamp())
        
        # timestamp of the end of the interval
        ts_end = datetime.strptime(f'31.12.{year} 23:59:59,99',
                                   '%d.%m.%Y %H:%M:%S,%f')
        ts_end = min(int(time.time()), int(ts_end.timestamp()))
        
        print('===== {subreddit} [{year}] =====')
        dfs = get_submissions(subreddit, ts_start, ts_end)
        df  = pd.concat(dfs.values(),
                        sort=True, ignore_index=True)
        
        file_out = f'{subreddit}_{year}.csv'
        if (len(df) > 0):
            df = df[(df['created_utc'] >= ts_start) &
                    (df['created_utc'] <= ts_end)]
            df.to_csv(f'{data_out}/{file_out}')
            print(f'\'{file_out}\' successfully saved!\n')
        else:
            print(f'\'{file_out}\' has no items!')

===== inthenews [2018] =====
Cycle    1: 2018-08-09 03:17:56 ... 1000 items done in   21 seconds!
Cycle    2: 2018-07-13 18:34:47 ... 1000 items done in   13 seconds!
Cycle    3: 2018-06-20 02:23:12 ... 1000 items done in    5 seconds!
Cycle    4: 2018-05-28 08:21:10 ... 1000 items done in   11 seconds!
Cycle    5: 2018-05-06 05:10:48 ... 1000 items done in    9 seconds!
Cycle    6: 2018-04-14 03:05:13 ... 1000 items done in    6 seconds!
Cycle    7: 2018-03-26 14:09:27 ... 1000 items done in   14 seconds!
Cycle    8: 2018-03-10 10:43:44 ... 1000 items done in    7 seconds!
Cycle    9: 2018-02-21 02:54:25 ... 1000 items done in    6 seconds!
Cycle   10: 2018-02-06 19:34:26 ... 1000 items done in    6 seconds!
Cycle   11: 2018-01-19 15:45:50 ... 1000 items done in    7 seconds!
Cycle   12: 2018-01-02 22:17:10 ... 1000 items done in    8 seconds!

The whole process took 113 seconds!
'inthenews_2018.csv' successfully saved!

===== savedyouaclick [2018] =====
Cycle    1: 2018-08-09 03:19:5

#### [01.01.201x - 31.12.201x] | x = {0 .. 7}

In [5]:
for year in list(np.arange(2017, 2009, -1)):
    for subreddit in ['inthenews', 'savedyouaclick']:  # news
        # timestamp of the start of the interval
        ts_start = datetime.strptime(f'01.01.{year} 00:00:00,00',
                                     '%d.%m.%Y %H:%M:%S,%f')
        ts_start = int(ts_start.timestamp())
        
        # timestamp of the end of the interval
        ts_end = datetime.strptime(f'31.12.{year} 23:59:59,99',
                                   '%d.%m.%Y %H:%M:%S,%f')
        ts_end = min(int(time.time()), int(ts_end.timestamp()))
        
        print(f'===== {subreddit} [{year}] =====')
        dfs = get_submissions(subreddit, ts_start, ts_end)
        df  = pd.concat(dfs.values(),
                        sort=True, ignore_index=True)
        
        file_out = f'{subreddit}_{year}.csv'
        if (len(df) > 0):
            df = df[(df['created_utc'] >= ts_start) &
                    (df['created_utc'] <= ts_end)]
            df.to_csv(f'{data_out}/{file_out}')
            print(f'\'{file_out}\' successfully saved!\n')
        else:
            print(f'\'{file_out}\' has no items!\n')

### /r/news...

In [ ]:
# timestamp of the start of the interval
ts_start = datetime.strptime('01.01.2000 00:00:00,00',
                             '%d.%m.%Y %H:%M:%S,%f')
ts_start = int(ts_start.timestamp())

# timestamp of the end of the interval
ts_end = datetime.strptime('31.12.2017 23:59:59,99',
                           '%d.%m.%Y %H:%M:%S,%f')
ts_end = int(ts_end.timestamp())

cycle = 1  # cycle index
time_total = 0
ts_current = min(int(time.time()), ts_end)

In [10]:
while ts_current > ts_start:
    time_start = time.time()  # start timestamp of the cycle

    print(f'Cycle {cycle:>4d}: {timestamp_to_utc(ts_current)} ...', end=' ')

    try:
        response = request.urlopen(f'https://api.pushshift.io/reddit/submission/search/' \
                                   f'?subreddit=news&before={ts_current}&size={maxsize}')
        data = response.read()
        data = json.loads(data)
        # data = get_submissions('news', before=ts_current)
    except:
        print('Error encountered!')
        time_end = time.time()  # end timestamp of the cycle
        time_total += int(time_end - time_start)
        continue

    df_cycle = json_normalize(data['data'])
    if len(df_cycle) > 0:
        file_out = f'news_cycle_{cycle:05d}.csv'
        df_cycle.to_csv(f'{data_out}/{file_out}')
#         print('\'{}\' successfully saved!\n'.format(file_out))
    else:
        print(f'\'{file_out}\' has no items!\n')
        time_end = time.time()  # end timestamp of the cycle
        time_total += int(time_end - time_start)
        break
        
    ts_current = df_cycle['created_utc'].iloc[-1]

    time_end = time.time()  # end timestamp of the cycle
    print(f'{len(df_cycle):>4d} items done in {int(time_end - time_start):>4d} seconds!')

    cycle += 1
    time_total += int(time_end - time_start)

print(f'\nThe whole process took {time_total} seconds!')
# print('Last timestamp: {}\n\n'.format(datetime.fromtimestamp(ts_current) \
#                                               .strftime('%Y-%m-%d %H:%M:%S')))

Cycle    1: 2017-12-31 23:59:59 ... 1000 items done in   12 seconds!
Cycle    2: 2017-12-30 20:13:17 ... 1000 items done in    7 seconds!
Cycle    3: 2017-12-30 05:31:32 ... 1000 items done in   21 seconds!
Cycle    4: 2017-12-29 12:29:53 ... 1000 items done in   11 seconds!
Cycle    5: 2017-12-28 23:47:51 ... 1000 items done in   11 seconds!
Cycle    6: 2017-12-28 10:25:11 ... 1000 items done in   15 seconds!
Cycle    7: 2017-12-27 20:22:54 ... 1000 items done in   10 seconds!
Cycle    8: 2017-12-27 09:23:56 ... 1000 items done in   10 seconds!
Cycle    9: 2017-12-26 16:26:37 ... 1000 items done in    7 seconds!
Cycle   10: 2017-12-26 06:07:21 ... 1000 items done in    4 seconds!
Cycle   11: 2017-12-25 07:06:54 ... 1000 items done in    5 seconds!
Cycle   12: 2017-12-24 04:32:26 ... 1000 items done in    7 seconds!
Cycle   13: 2017-12-23 08:08:28 ... 1000 items done in    4 seconds!
Cycle   14: 2017-12-22 14:27:25 ... 1000 items done in    5 seconds!
Cycle   15: 2017-12-22 05:48:07 ..

Cycle  120: 2017-10-30 11:11:47 ... 1000 items done in    6 seconds!
Cycle  121: 2017-10-30 01:10:33 ... 1000 items done in   11 seconds!
Cycle  122: 2017-10-29 06:56:09 ... 1000 items done in   10 seconds!
Cycle  123: 2017-10-28 13:09:23 ... 1000 items done in   10 seconds!
Cycle  124: 2017-10-27 22:29:00 ... 1000 items done in   12 seconds!
Cycle  125: 2017-10-27 13:05:56 ... 1000 items done in    4 seconds!
Cycle  126: 2017-10-27 04:33:26 ... 1000 items done in    5 seconds!
Cycle  127: 2017-10-26 16:26:27 ... 1000 items done in    6 seconds!
Cycle  128: 2017-10-26 08:56:05 ... 1000 items done in    4 seconds!
Cycle  129: 2017-10-25 19:34:19 ... 1000 items done in    5 seconds!
Cycle  130: 2017-10-25 11:45:16 ... 1000 items done in   16 seconds!
Cycle  131: 2017-10-25 03:48:27 ... 1000 items done in    6 seconds!
Cycle  132: 2017-10-24 16:14:30 ... 1000 items done in   10 seconds!
Cycle  133: 2017-10-24 09:07:57 ... 1000 items done in    5 seconds!
Cycle  134: 2017-10-23 20:07:32 ..

Cycle  239: 2017-08-30 12:34:06 ... 1000 items done in    4 seconds!
Cycle  240: 2017-08-30 02:31:26 ... 1000 items done in    4 seconds!
Cycle  241: 2017-08-29 14:12:04 ... 1000 items done in    4 seconds!
Cycle  242: 2017-08-29 06:25:44 ... 1000 items done in    4 seconds!
Cycle  243: 2017-08-28 16:52:55 ... 1000 items done in    5 seconds!
Cycle  244: 2017-08-28 08:22:10 ... 1000 items done in   12 seconds!
Cycle  245: 2017-08-27 13:57:01 ... 1000 items done in    4 seconds!
Cycle  246: 2017-08-26 15:06:47 ... 1000 items done in    4 seconds!
Cycle  247: 2017-08-26 01:28:01 ... 1000 items done in   22 seconds!
Cycle  248: 2017-08-25 12:27:36 ... 1000 items done in   16 seconds!
Cycle  249: 2017-08-24 22:18:42 ... 1000 items done in   10 seconds!
Cycle  250: 2017-08-24 12:17:28 ... 1000 items done in    8 seconds!
Cycle  251: 2017-08-24 01:26:32 ... 1000 items done in    9 seconds!
Cycle  252: 2017-08-23 13:47:46 ... 1000 items done in    5 seconds!
Cycle  253: 2017-08-23 06:39:44 ..

#### Get the timestamp of the last cycle in the previous iteration and continue from there...

In [13]:
while ts_current > ts_start:
    time_start = time.time()  # start timestamp of the cycle

    print(f'Cycle {cycle:>4d}: {timestamp_to_utc(ts_current)} ...', end=' ')

    try:
        response = request.urlopen(f'https://api.pushshift.io/reddit/submission/search/' \
                                   f'?subreddit={news}&before={ts_current}&size={maxsize}')
        data = response.read()
        data = json.loads(data)
        # data = get_submissions('news', before=ts_current)
    except:
        print('Error encountered!')
        time_end = time.time()  # end timestamp of the cycle
        time_total += int(time_end - time_start)
        continue

    df_cycle = json_normalize(data['data'])
    if len(df_cycle) > 0:
        file_out = f'news_cycle_{cycle:05d}.csv'
        df_cycle.to_csv(f'{data_out}/{file_out}')
#         print('\'{}\' successfully saved!\n'.format(file_out))
    else:
        print(f'\'{file_out}\' has no items!\n')
        time_end = time.time()  # end timestamp of the cycle
        time_total += int(time_end - time_start)
        break
        
    ts_current = df_cycle['created_utc'].iloc[-1]

    time_end = time.time()  # end timestamp of the cycle
    print(f'{len(df_cycle):>4d} items done in {int(time_end - time_start):>4d} seconds!')

    cycle += 1
    time_total += int(time_end - time_start)

print(f'\nThe whole process took {time_total} seconds!')
# print('Last timestamp: {}\n\n'.format(datetime.fromtimestamp(ts_current) \
#                                               .strftime('%Y-%m-%d %H:%M:%S')))

Cycle  271: 2017-08-14 12:21:08 ... 1000 items done in    6 seconds!
Cycle  272: 2017-08-13 23:23:03 ... 1000 items done in    6 seconds!
Cycle  273: 2017-08-13 03:07:35 ... 1000 items done in    7 seconds!
Cycle  274: 2017-08-12 13:30:41 ... 1000 items done in    6 seconds!
Cycle  275: 2017-08-11 22:52:08 ... 1000 items done in    7 seconds!
Cycle  276: 2017-08-11 12:09:48 ... 1000 items done in    6 seconds!
Cycle  277: 2017-08-11 00:11:12 ... 1000 items done in    7 seconds!
Cycle  278: 2017-08-10 13:22:20 ... 1000 items done in    7 seconds!
Cycle  279: 2017-08-10 04:46:30 ... 1000 items done in    6 seconds!
Cycle  280: 2017-08-09 14:36:11 ... 1000 items done in    6 seconds!
Cycle  281: 2017-08-09 07:07:40 ... 1000 items done in    6 seconds!
Cycle  282: 2017-08-08 17:20:43 ... 1000 items done in    7 seconds!
Cycle  283: 2017-08-08 09:27:05 ... 1000 items done in    6 seconds!
Cycle  284: 2017-08-07 19:44:53 ... 1000 items done in    6 seconds!
Cycle  285: 2017-08-07 08:21:10 ..

Cycle  390: 2017-06-16 19:24:29 ... 1000 items done in    7 seconds!
Cycle  391: 2017-06-16 11:26:49 ... 1000 items done in    6 seconds!
Cycle  392: 2017-06-15 23:41:44 ... 1000 items done in    6 seconds!
Cycle  393: 2017-06-15 13:17:54 ... 1000 items done in    7 seconds!
Cycle  394: 2017-06-15 06:43:29 ... 1000 items done in    7 seconds!
Cycle  395: 2017-06-14 18:46:11 ... 1000 items done in    7 seconds!
Cycle  396: 2017-06-14 11:49:10 ... 1000 items done in   11 seconds!
Cycle  397: 2017-06-14 03:03:59 ... 1000 items done in    8 seconds!
Cycle  398: 2017-06-13 14:18:38 ... 1000 items done in    5 seconds!
Cycle  399: 2017-06-13 07:49:27 ... 1000 items done in    7 seconds!
Cycle  400: 2017-06-12 17:33:27 ... 1000 items done in    6 seconds!
Cycle  401: 2017-06-12 09:47:36 ... Error encountered!
Cycle  401: 2017-06-12 09:47:36 ... Error encountered!
Cycle  401: 2017-06-12 09:47:36 ... 1000 items done in    2 seconds!
Cycle  402: 2017-06-11 19:41:05 ... Error encountered!
Cycle  

Cycle  506: 2017-04-25 09:45:37 ... 1000 items done in    6 seconds!
Cycle  507: 2017-04-25 00:28:19 ... 1000 items done in    7 seconds!
Cycle  508: 2017-04-24 14:15:50 ... 1000 items done in    5 seconds!
Cycle  509: 2017-04-24 08:19:59 ... 1000 items done in    6 seconds!
Cycle  510: 2017-04-23 17:15:39 ... 1000 items done in    7 seconds!
Cycle  511: 2017-04-22 21:35:21 ... 1000 items done in    9 seconds!
Cycle  512: 2017-04-22 09:49:53 ... 1000 items done in    6 seconds!
Cycle  513: 2017-04-21 20:34:40 ... 1000 items done in    6 seconds!
Cycle  514: 2017-04-21 12:11:18 ... 1000 items done in   10 seconds!
Cycle  515: 2017-04-21 05:13:28 ... 1000 items done in    5 seconds!
Cycle  516: 2017-04-20 17:59:30 ... 1000 items done in    6 seconds!
Cycle  517: 2017-04-20 11:03:08 ... 1000 items done in    6 seconds!
Cycle  518: 2017-04-20 03:22:18 ... 1000 items done in    6 seconds!
Cycle  519: 2017-04-19 17:18:00 ... 1000 items done in    5 seconds!
Cycle  520: 2017-04-19 11:16:47 ..

Cycle  624: 2017-03-07 12:14:06 ... 1000 items done in    9 seconds!
Cycle  625: 2017-03-07 06:28:37 ... 1000 items done in   11 seconds!
Cycle  626: 2017-03-06 18:13:40 ... 1000 items done in    6 seconds!
Cycle  627: 2017-03-06 10:45:28 ... 1000 items done in   12 seconds!
Cycle  628: 2017-03-06 03:18:38 ... 1000 items done in   11 seconds!
Cycle  629: 2017-03-05 09:51:08 ... 1000 items done in    6 seconds!
Cycle  630: 2017-03-04 15:44:50 ... 1000 items done in    6 seconds!
Cycle  631: 2017-03-04 05:30:30 ... 1000 items done in    5 seconds!
Cycle  632: 2017-03-03 17:43:05 ... 1000 items done in    7 seconds!
Cycle  633: 2017-03-03 10:32:26 ... 1000 items done in    5 seconds!
Cycle  634: 2017-03-03 02:52:16 ... 1000 items done in    5 seconds!
Cycle  635: 2017-03-02 16:16:54 ... 1000 items done in    5 seconds!
Cycle  636: 2017-03-02 09:43:12 ... 1000 items done in    5 seconds!
Cycle  637: 2017-03-02 02:17:11 ... 1000 items done in    7 seconds!
Cycle  638: 2017-03-01 14:32:48 ..

Cycle  743: 2017-01-19 05:01:34 ... 1000 items done in    5 seconds!
Cycle  744: 2017-01-18 18:11:07 ... 1000 items done in    5 seconds!
Cycle  745: 2017-01-18 11:09:35 ... 1000 items done in    6 seconds!
Cycle  746: 2017-01-18 04:08:31 ... 1000 items done in    5 seconds!
Cycle  747: 2017-01-17 17:10:16 ... 1000 items done in    4 seconds!
Cycle  748: 2017-01-17 09:57:26 ... 1000 items done in    5 seconds!
Cycle  749: 2017-01-16 23:30:22 ... 1000 items done in    4 seconds!
Cycle  750: 2017-01-16 13:13:04 ... 1000 items done in    4 seconds!
Cycle  751: 2017-01-16 07:03:54 ... 1000 items done in    5 seconds!
Cycle  752: 2017-01-15 14:44:40 ... 1000 items done in    5 seconds!
Cycle  753: 2017-01-14 19:36:46 ... 1000 items done in    6 seconds!
Cycle  754: 2017-01-14 06:14:39 ... 1000 items done in    5 seconds!
Cycle  755: 2017-01-13 16:35:39 ... 1000 items done in    5 seconds!
Cycle  756: 2017-01-13 09:07:39 ... 1000 items done in    4 seconds!
Cycle  757: 2017-01-12 22:03:52 ..

...

Cycle 2766: 2013-10-08 05:06:21 ... 1000 items done in    4 seconds!
Cycle 2767: 2013-10-07 10:29:17 ... 1000 items done in    4 seconds!
Cycle 2768: 2013-10-06 03:47:41 ... 1000 items done in    4 seconds!
Cycle 2769: 2013-10-05 02:39:25 ... 1000 items done in    3 seconds!
Cycle 2770: 2013-10-04 08:34:23 ... 1000 items done in    3 seconds!
Cycle 2771: 2013-10-03 16:45:18 ... 1000 items done in    3 seconds!
Cycle 2772: 2013-10-02 23:31:16 ... 1000 items done in    3 seconds!
Cycle 2773: 2013-10-02 08:17:25 ... 1000 items done in    3 seconds!
Cycle 2774: 2013-10-01 12:49:56 ... 1000 items done in    3 seconds!
Cycle 2775: 2013-09-30 17:01:24 ... 1000 items done in    3 seconds!
Cycle 2776: 2013-09-29 15:48:23 ... 1000 items done in    3 seconds!
Cycle 2777: 2013-09-28 06:10:50 ... 1000 items done in    3 seconds!
Cycle 2778: 2013-09-27 08:57:26 ... 1000 items done in    8 seconds!
Cycle 2779: 2013-09-26 13:41:03 ... 1000 items done in    4 seconds!
Cycle 2780: 2013-09-25 18:28:56 ..

Cycle 2885: 2013-06-19 05:34:30 ... 1000 items done in    3 seconds!
Cycle 2886: 2013-06-18 07:00:00 ... 1000 items done in    4 seconds!
Cycle 2887: 2013-06-17 08:35:23 ... 1000 items done in    3 seconds!
Cycle 2888: 2013-06-15 10:36:33 ... 1000 items done in    3 seconds!
Cycle 2889: 2013-06-14 10:25:47 ... 1000 items done in    4 seconds!
Cycle 2890: 2013-06-13 13:03:15 ... 1000 items done in    9 seconds!
Cycle 2891: 2013-06-12 14:27:50 ... 1000 items done in    3 seconds!
Cycle 2892: 2013-06-11 16:53:31 ... 1000 items done in    3 seconds!
Cycle 2893: 2013-06-10 19:28:27 ... 1000 items done in    3 seconds!
Cycle 2894: 2013-06-09 14:29:09 ... 1000 items done in    4 seconds!
Cycle 2895: 2013-06-08 01:05:31 ... 1000 items done in    3 seconds!
Cycle 2896: 2013-06-07 05:41:32 ... 1000 items done in    3 seconds!
Cycle 2897: 2013-06-06 08:43:46 ... 1000 items done in    4 seconds!
Cycle 2898: 2013-06-05 13:04:09 ... 1000 items done in    4 seconds!
Cycle 2899: 2013-06-04 16:56:26 ..

Cycle 3004: 2012-11-30 12:26:47 ... 1000 items done in    3 seconds!
Cycle 3005: 2012-11-28 07:14:23 ... 1000 items done in    4 seconds!
Cycle 3006: 2012-11-25 03:45:27 ... 1000 items done in    3 seconds!
Cycle 3007: 2012-11-21 20:48:52 ... 1000 items done in    4 seconds!
Cycle 3008: 2012-11-19 10:14:26 ... 1000 items done in    3 seconds!
Cycle 3009: 2012-11-16 12:11:31 ... 1000 items done in    4 seconds!
Cycle 3010: 2012-11-14 06:11:50 ... 1000 items done in    3 seconds!
Cycle 3011: 2012-11-11 20:41:14 ... 1000 items done in    3 seconds!
Cycle 3012: 2012-11-09 04:56:43 ... 1000 items done in    3 seconds!
Cycle 3013: 2012-11-06 21:09:15 ... 1000 items done in    3 seconds!
Cycle 3014: 2012-11-03 15:38:24 ... 1000 items done in    3 seconds!
Cycle 3015: 2012-10-31 20:59:59 ... 1000 items done in    4 seconds!
Cycle 3016: 2012-10-29 19:21:33 ... 1000 items done in    3 seconds!
Cycle 3017: 2012-10-26 17:04:30 ... 1000 items done in    4 seconds!
Cycle 3018: 2012-10-24 02:28:42 ..

Cycle 3123: 2012-01-06 15:28:51 ... 1000 items done in    3 seconds!
Cycle 3124: 2012-01-03 00:38:37 ... 1000 items done in    3 seconds!
Cycle 3125: 2011-12-29 21:43:46 ... 1000 items done in    4 seconds!
Cycle 3126: 2011-12-25 10:09:21 ... 1000 items done in    3 seconds!
Cycle 3127: 2011-12-21 17:02:59 ... 1000 items done in    3 seconds!
Cycle 3128: 2011-12-18 03:58:12 ... 1000 items done in    8 seconds!
Cycle 3129: 2011-12-14 20:29:45 ... 1000 items done in    3 seconds!
Cycle 3130: 2011-12-11 07:11:24 ... 1000 items done in    4 seconds!
Cycle 3131: 2011-12-08 09:44:58 ... 1000 items done in    3 seconds!
Cycle 3132: 2011-12-05 00:18:50 ... 1000 items done in    3 seconds!
Cycle 3133: 2011-12-01 04:08:36 ... 1000 items done in    4 seconds!
Cycle 3134: 2011-11-28 07:43:00 ... 1000 items done in    3 seconds!
Cycle 3135: 2011-11-24 05:41:29 ... 1000 items done in    4 seconds!
Cycle 3136: 2011-11-21 16:58:29 ... 1000 items done in    3 seconds!
Cycle 3137: 2011-11-17 18:25:19 ..

Cycle 3242: 2009-09-12 03:52:20 ... 1000 items done in    3 seconds!
Cycle 3243: 2009-08-30 21:29:07 ... 1000 items done in    5 seconds!
Cycle 3244: 2009-08-18 02:53:20 ... 1000 items done in    3 seconds!
Cycle 3245: 2009-08-06 00:01:34 ... 1000 items done in    3 seconds!
Cycle 3246: 2009-07-24 07:06:47 ... 1000 items done in    4 seconds!
Cycle 3247: 2009-07-12 06:57:01 ... 1000 items done in    3 seconds!
Cycle 3248: 2009-06-30 14:48:17 ... 1000 items done in    2 seconds!
Cycle 3249: 2009-06-16 18:39:52 ... 1000 items done in    3 seconds!
Cycle 3250: 2009-06-01 16:46:57 ... 1000 items done in    3 seconds!
Cycle 3251: 2009-05-13 04:00:49 ... 1000 items done in    3 seconds!
Cycle 3252: 2009-04-27 19:00:15 ... 1000 items done in    4 seconds!
Cycle 3253: 2009-04-10 22:36:40 ... 1000 items done in    3 seconds!
Cycle 3254: 2009-03-24 21:58:48 ... 1000 items done in    3 seconds!
Cycle 3255: 2009-03-07 01:43:22 ... 1000 items done in    3 seconds!
Cycle 3256: 2009-02-13 15:30:54 ..